In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2  
from tensorflow.keras.callbacks import TensorBoard

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
filename=r"D:\seriestesteapp\super_loja.csv"

In [4]:
df = pd.read_csv(filename,sep=";",encoding="latin-1")
df.dropna(inplace=True)
df[['Vendas','Desconto','Lucro']] = df[['Vendas','Desconto','Lucro']].applymap(lambda k: float(str(k).replace(",",".")))
df['Data do pedido'] = pd.to_datetime(df['Data do pedido'],errors='coerce')

In [5]:

columns_to_keep = ['Modo de envio', 'Segmento', 'Cidade', 'Estado', 'País/Região', 'Região',
                   'Categoria', 'Sub-categoria', 'Vendas', 'Quantidade', 'Desconto', 'Lucro']
df_filtered = df[columns_to_keep]


X = df_filtered.drop(columns=['Vendas'])
y = df_filtered['Vendas']


categorical_columns = ['Modo de envio', 'Segmento', 'Cidade', 'Estado', 'País/Região', 'Região',
                       'Categoria', 'Sub-categoria']
encoder = OneHotEncoder(sparse=False, drop='first')
X_encoded = encoder.fit_transform(X[categorical_columns])

encoded_columns = encoder.get_feature_names_out(input_features=categorical_columns)
X_encoded_df = pd.DataFrame(X_encoded, columns=encoded_columns, index=X.index)

X = pd.concat([X.drop(columns=categorical_columns), X_encoded_df], axis=1)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


modelo = Sequential()
modelo.add(Dense(units=1512, activation='relu', input_dim=X_scaled.shape[1], kernel_regularizer=l2(0.001))) 
modelo.add(Dropout(0.4))
modelo.add(Dense(units=256, activation='relu', kernel_regularizer=l2(0.001)))
modelo.add(Dropout(0.4))
modelo.add(Dense(units=128, activation='relu', kernel_regularizer=l2(0.001)))
modelo.add(Dense(units=64, activation='relu', kernel_regularizer=l2(0.001)))
modelo.add(Dense(units=32, activation='relu', kernel_regularizer=l2(0.001)))
modelo.add(Dense(units=1, activation='linear'))


optimizer = Adam(learning_rate=0.0001)
modelo.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mae'])


tensorboard = TensorBoard(log_dir=r'D:\seriestesteapp\logs', histogram_freq=1, write_graph=True, write_images=True)


resultado = modelo.fit(X_train, y_train, epochs=450, batch_size=64, verbose=1, validation_split=0.2, callbacks=[tensorboard])


previsoes = modelo.predict(X_test)


for index, valor_real in enumerate(y_test):
    previsao = previsoes[index][0]
    print(f'Índice: {index}, Valor Real: {valor_real:.2f}, Previsão: {previsao:.2f}')


Epoch 1/450
3/3 [==============================] - 3s 755ms/step - loss: 15971980.0000 - mae: 2299.8108 - val_loss: 18916656.0000 - val_mae: 2237.6372
Epoch 2/450
3/3 [==============================] - 1s 594ms/step - loss: 15971020.0000 - mae: 2299.6025 - val_loss: 18915562.0000 - val_mae: 2237.3853
Epoch 3/450
3/3 [==============================] - 2s 877ms/step - loss: 15969389.0000 - mae: 2299.3630 - val_loss: 18914528.0000 - val_mae: 2237.1387
Epoch 4/450
3/3 [==============================] - 1s 590ms/step - loss: 15968367.0000 - mae: 2299.1255 - val_loss: 18913518.0000 - val_mae: 2236.8853
Epoch 5/450
3/3 [==============================] - 1s 559ms/step - loss: 15967165.0000 - mae: 2298.8787 - val_loss: 18912414.0000 - val_mae: 2236.6221
Epoch 6/450
3/3 [==============================] - 1s 420ms/step - loss: 15965420.0000 - mae: 2298.5452 - val_loss: 18911244.0000 - val_mae: 2236.3425
Epoch 7/450
3/3 [==============================] - 1s 437ms/step - loss: 15963977.0000 - mae: 

In [ ]:
%tensorboard --logdir ./logs
